In [ ]:
import scanpy as sc
import scarches as sca
import numpy as np

In [ ]:
sc.set_figure_params(vector_friendly=True)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
plt.rcParams["axes.grid"] = False
plt.rcParams["figure.dpi"] = 140
font = {'family' : 'Arial',
        'size'   : 14}

matplotlib.rc('font', **font)
matplotlib.rc('ytick', labelsize=14)
matplotlib.rc('xtick', labelsize=14)

In [ ]:
def filter_set_scores(scores, adata, filter_v=2.31):
    for k in scores:
        print(k)
        mask = np.abs(scores[k]['bf']) > filter_v
        s = sum(mask)
        if s > 0:
            sort = np.argsort(np.abs(scores[k]['bf'])[mask])[::-1]
            enriched_terms = adata.uns['terms'][mask][sort]
            
            print(enriched_terms)
            print(scores[k]['bf'][mask][sort])
            
            for term in enriched_terms:
                adata.obs[term] = adata.obsm['X_cvae'][:, adata.uns['active_terms'] == term]

### Pbmc Kang

In [ ]:
adata = sc.read('kang_pbmc_integrated.h5ad')

In [ ]:
adata.obs['condition_merged'] = adata.obs['condition'].tolist()
adata.obs['condition_merged'][adata.obs['condition_merged'].astype(str)=='nan'] = 'control'
adata.strings_to_categoricals()

adata.obs.batch_join = adata.obs.batch_join.astype(str)
adata.obs.batch_join[adata.obs.batch_join=='0'] = 'PBMC (reference)'
adata.obs.batch_join[adata.obs.batch_join=='1'] = 'Kang (query)'

adata.obs['condition_joint'] = adata.obs.condition.astype(str)
adata.obs['condition_joint'][adata.obs['condition_joint'].astype(str)=='nan']='control (reference)'
adata.obs['condition_joint'][adata.obs['condition_joint']=='control']='control (query)'
adata.obs['condition_joint'][adata.obs['condition_joint']=='stimulated']='stimulated (query)'

In [ ]:
intr_cvae = sca.models.EXPIMAP.load('q_intr_cvae_nolog_alpha_kl_0_5_0_1_sd_2020', adata)

In [ ]:
adata.uns['active_terms'] = adata.uns['terms'][intr_cvae.model.decoder.nonzero_terms()]

In [ ]:
directions = intr_cvae.latent_directions(method="sum")

In [ ]:
directions = adata.uns['directions']


In [ ]:
print(directions)

adata.obsm['X_cvae'] *= directions[intr_cvae.nonzero_terms()]


In [ ]:
scores_cond = intr_cvae.latent_enrich('condition_merged', comparison="control", directions_key=directions, adata=adata)

In [ ]:
scores_cond = adata.uns['bf_scores']

In [ ]:
filter_set_scores(scores_cond, adata, filter_v=0.5)

In [ ]:
adata.obs['INTERFERON_ALPHA_BETA_SIGNALING'] = adata.obs['INTERFERON_ALPHA_BETA_SIGNALIN']

In [ ]:
sc.pl.scatter(adata, x='INTERFERON_ALPHA_BETA_SIGNALING', y='INTERFERON_SIGNALING', 
              color='condition_joint', size=10, title='Condition', 
              groups=['control (reference)', 'control (query)', 'stimulated (query)'],
              save='_pbmc_kang_interferon.png', palette=['steelblue', 'skyblue', 'red'])

In [ ]:
scores_ct = intr_cvae.latent_enrich('cell_type_joint', directions_key=directions, n_sample=7000, adata=adata)

In [ ]:
scores_ct = adata.uns['bf_scores']

In [ ]:
filter_set_scores(scores_ct, adata, filter_v=1.5)

In [ ]:
for cat in adata.obs.cell_type_joint[adata.obs.batch_join == 'Kang (query)'].unique():
    print(cat)
    adata_ct = adata[adata.obs.cell_type_joint == cat]
    scores_c_ct = intr_cvae.latent_enrich('condition_merged', comparison="control", directions_key=directions, adata=adata_ct)
    scores_c_ct = adata.uns['bf_scores']
    filter_set_scores(scores_c_ct, adata)

In [ ]:
sc.pl.scatter(adata, x='NUCLEOTIDE_BINDING_DOMAIN_LEUC', y='FACTORS_INVOLVED_IN_MEGAKARYOC', 
              size=22)

In [ ]:
adata.obs.cell_type_joint[adata.obs.batch_join == 'Kang (query)'].unique()

In [ ]:
adata.obs.cell_type_joint.cat.categories

### Figure

In [ ]:
matplotlib.rcParams['axes.labelsize'] = 15.7

In [ ]:
condition = 'condition_joint'
condition_loc = 'lower left'
groups = ['control (reference)', 'control (query)', 'stimulated (query)']
size = 10
palette=['steelblue', 'skyblue', 'red']

In [ ]:
fig, axs = plt.subplots(5, 3)

In [ ]:
adata.obs['MHC_CLASS_II_ANTIGEN_PRESENTATION'] = adata.obs['MHC_CLASS_II_ANTIGEN_PRESENTAT']
adata.obs['SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR'] = adata.obs['SIGNALING_BY_THE_B_CELL_RECEPT']

In [ ]:
cell_mask = ~np.in1d(adata.obs['cell_type_joint'],['B', 'CD10+ B cells'])

In [ ]:
adata.obs['cell_type_b'] = adata.obs['cell_type_joint'].astype('O')
adata.obs['cell_type_b'][cell_mask] = 'others'

In [ ]:
sc.pl.scatter(adata, x='MHC_CLASS_II_ANTIGEN_PRESENTATION', y='SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR', 
              color='cell_type_b')

In [ ]:
for i, cat in enumerate(adata.obs['cell_type_b'].cat.categories):
    if cat=='others':
        adata.uns['cell_type_b_colors'][i] = 'gray'
    else:
        idx = adata.obs['cell_type_joint'].cat.categories == cat
        adata.uns['cell_type_b_colors'][i] = np.array(adata.uns['cell_type_joint_colors'])[idx][0]

In [ ]:
sc.pl.scatter(adata, x='SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR', y='MHC_CLASS_II_ANTIGEN_PRESENTATION', 
              color='cell_type_b', size=size, title='B cells', legend_loc='lower left', ax=axs[0,0],
              groups=['others', 'B', 'CD10+ B cells'])

In [ ]:
sc.pl.scatter(adata, x='SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR', y='INTERFERON_SIGNALING', 
              color='cell_type_b', size=size, title='B cells', legend_loc='lower left', ax=axs[0,1],
              groups=['others', 'B', 'CD10+ B cells'])

In [ ]:
sc.pl.scatter(adata, x='SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR', y='INTERFERON_SIGNALING', 
              color=condition, size=size, title='B cells condition', legend_loc=condition_loc, groups=groups, ax=axs[0,2], 
              palette=palette)

In [ ]:
fig.set_size_inches(25, 28)

In [ ]:
fig.tight_layout() 

In [ ]:
fig

In [ ]:
adata.obs['METABOLISM_OF_LIPIDS_AND_LIPOPROTEINS'] = adata.obs['METABOLISM_OF_LIPIDS_AND_LIPOP']
adata.obs['METABOLISM_OF_AMINO_ACIDS_AND_DERIVATIVES'] = adata.obs['METABOLISM_OF_AMINO_ACIDS_AND_']
adata.obs['IMMUNOREGULATORY_INTERACTIONS_LYMPHOID_-_NON_LYMPHOID_CELL'] = adata.obs['IMMUNOREGULATORY_INTERACTIONS_']
adata.obs['PLATELET_ACTIVATION_SIGNALING_AND_AGGREGATION'] = adata.obs['PLATELET_ACTIVATION_SIGNALING_']

In [ ]:
cell_mask = ~np.in1d(adata.obs['cell_type_joint'],['CD16+ Monocytes', 'CD14+ Monocytes'])

In [ ]:
adata.obs['cell_type_m'] = adata.obs['cell_type_joint'].astype('O')
adata.obs['cell_type_m'][cell_mask] = 'others'

In [ ]:
sc.pl.scatter(adata, x='METABOLISM_OF_LIPIDS_AND_LIPOPROTEINS', y='METABOLISM_OF_AMINO_ACIDS_AND_DERIVATIVES', 
              color='cell_type_m')

In [ ]:
for i, cat in enumerate(adata.obs['cell_type_m'].cat.categories):
    if cat=='others':
        adata.uns['cell_type_m_colors'][i] = 'gray'
    else:
        idx = adata.obs['cell_type_joint'].cat.categories == cat
        adata.uns['cell_type_m_colors'][i] = np.array(adata.uns['cell_type_joint_colors'])[idx][0]

In [ ]:
sc.pl.scatter(adata, x='METABOLISM_OF_LIPIDS_AND_LIPOPROTEINS', y='PLATELET_ACTIVATION_SIGNALING_AND_AGGREGATION', 
              color='cell_type_m', size=size, title='Monocytes', legend_loc='lower left', ax=axs[1,0],
              groups=['others', 'CD14+ Monocytes', 'CD16+ Monocytes'])

In [ ]:
sc.pl.scatter(adata, x='METABOLISM_OF_LIPIDS_AND_LIPOPROTEINS', y='INTERFERON_SIGNALING', 
              color='cell_type_m', size=size, title='Monocytes', legend_loc='lower left', ax=axs[1,1],
              groups=['others', 'CD14+ Monocytes', 'CD16+ Monocytes'])

In [ ]:
sc.pl.scatter(adata, x='METABOLISM_OF_LIPIDS_AND_LIPOPROTEINS', y='INTERFERON_SIGNALING', 
              color=condition, size=size, title='Monocytes condition', legend_loc=condition_loc, groups=groups, ax=axs[1,2],
              palette=palette)

In [ ]:
fig

In [ ]:
cell_mask = ~np.in1d(adata.obs['cell_type_joint'],['Monocyte-derived dendritic cells', 'DC'])

In [ ]:
adata.obs['cell_type_d'] = adata.obs['cell_type_joint'].astype('O')
adata.obs['cell_type_d'][cell_mask] = 'others'

In [ ]:
sc.pl.scatter(adata, x='MHC_CLASS_II_ANTIGEN_PRESENTATION', y='AMYLOIDS', color='cell_type_d')

In [ ]:
for i, cat in enumerate(adata.obs['cell_type_d'].cat.categories):
    if cat=='others':
        adata.uns['cell_type_d_colors'][i] = 'gray'
    else:
        idx = adata.obs['cell_type_joint'].cat.categories == cat
        adata.uns['cell_type_d_colors'][i] = np.array(adata.uns['cell_type_joint_colors'])[idx][0]

In [ ]:
sc.pl.scatter(adata, x='MHC_CLASS_II_ANTIGEN_PRESENTATION', y='AMYLOIDS', 
              color='cell_type_d', size=size, title='Dendritic cells', legend_loc='lower right', ax=axs[2,0],
              groups=['others', 'DC', 'Monocyte-derived dendritic cells'])

In [ ]:
sc.pl.scatter(adata, x='MHC_CLASS_II_ANTIGEN_PRESENTATION', y='INTERFERON_SIGNALING', 
              color='cell_type_d', size=size, title='Dendritic cells', legend_loc='lower right', ax=axs[2,1],
              groups=['others', 'DC', 'Monocyte-derived dendritic cells'])

In [ ]:
sc.pl.scatter(adata, x='MHC_CLASS_II_ANTIGEN_PRESENTATION', y='INTERFERON_SIGNALING', 
              color=condition, size=size, title='Dendritic cells condition', legend_loc=condition_loc, groups=groups, ax=axs[2,2],
              palette=palette)

In [ ]:
fig

In [ ]:
cell_mask = ~np.in1d(adata.obs['cell_type_joint'],['CD4+ T cells', 'CD8+ T cells', 'T', 'NKT cells'])

In [ ]:
adata.obs['cell_type_t'] = adata.obs['cell_type_joint'].astype('O')
adata.obs['cell_type_t'][cell_mask] = 'others'

In [ ]:
sc.pl.scatter(adata, x='ADAPTIVE_IMMUNE_SYSTEM', y='SIGNALING_BY_GPCR', 
              color='cell_type_t', groups=['others', 'CD4+ T cells', 'NKT cells', 'CD8+ T cells', 'T'])

In [ ]:
for i, cat in enumerate(adata.obs['cell_type_t'].cat.categories):
    if cat=='others':
        adata.uns['cell_type_t_colors'][i] = 'gray'
    else:
        idx = adata.obs['cell_type_joint'].cat.categories == cat
        adata.uns['cell_type_t_colors'][i] = np.array(adata.uns['cell_type_joint_colors'])[idx][0]

In [ ]:
sc.pl.scatter(adata, x='IMMUNE_SYSTEM', y='SIGNALING_BY_GPCR', 
              color='cell_type_t', size=size, title='T cells', legend_loc='lower right', ax=axs[3,0],
              groups=['others', 'CD4+ T cells', 'NKT cells', 'CD8+ T cells', 'T'])

In [ ]:
sc.pl.scatter(adata, x='IMMUNE_SYSTEM', y='INTERFERON_SIGNALING', 
              color='cell_type_t', size=size, title='T cells', legend_loc='lower right', ax=axs[3,1],
              groups=['others', 'CD4+ T cells', 'NKT cells', 'CD8+ T cells', 'T'])

In [ ]:
sc.pl.scatter(adata, x='IMMUNE_SYSTEM', y='INTERFERON_SIGNALING', 
              color=condition, size=size, title='T cells condition', legend_loc=condition_loc, groups=groups, ax=axs[3,2],
              palette=palette)

In [ ]:
fig

In [ ]:
cell_mask = ~np.in1d(adata.obs['cell_type_joint'],['NK cells'])

In [ ]:
adata.obs['cell_type_n'] = adata.obs['cell_type_joint'].astype('O')
adata.obs['cell_type_n'][cell_mask] = 'others'

In [ ]:
adata.obs['IMMUNOREGULATORY_INTERACTIONS_BETWEEN_A_LYMPHOID_AND_A_NON_LYMPHOID_CELL'] = adata.obs['IMMUNOREGULATORY_INTERACTIONS_']

In [ ]:
sc.pl.scatter(adata, x='IMMUNOREGULATORY_INTERACTIONS_BETWEEN_A_LYMPHOID_AND_A_NON_LYMPHOID_CELL', y='GPCR_LIGAND_BINDING', 
              color='cell_type_n')

In [ ]:
for i, cat in enumerate(adata.obs['cell_type_n'].cat.categories):
    if cat=='others':
        adata.uns['cell_type_n_colors'][i] = 'gray'
    else:
        idx = adata.obs['cell_type_joint'].cat.categories == cat
        adata.uns['cell_type_n_colors'][i] = np.array(adata.uns['cell_type_joint_colors'])[idx][0]

In [ ]:
sc.pl.scatter(adata, x='IMMUNOREGULATORY_INTERACTIONS_LYMPHOID_-_NON_LYMPHOID_CELL', y='GPCR_LIGAND_BINDING', 
              color='cell_type_n', size=size, title='NK cells', legend_loc='lower right', ax=axs[4,0],
              groups=['others', 'NK cells'])

In [ ]:
sc.pl.scatter(adata, x='IMMUNOREGULATORY_INTERACTIONS_LYMPHOID_-_NON_LYMPHOID_CELL', y='INTERFERON_SIGNALING', 
              color='cell_type_n', size=size, title='NK cells', legend_loc='lower right', ax=axs[4,1],
              groups=['others', 'NK cells'])

In [ ]:
sc.pl.scatter(adata, x='IMMUNOREGULATORY_INTERACTIONS_LYMPHOID_-_NON_LYMPHOID_CELL', y='INTERFERON_SIGNALING', 
              color=condition, size=size, title='NK cells condition', legend_loc=condition_loc, groups=groups, ax=axs[4,2])

In [ ]:
fig

In [ ]:
fig.savefig('pbmc_kang_cell_types_merged_nole_new.png', bbox_inches = "tight")